# Notebook 2: Creating Items for Religious Communities
This notebook implements the second step of the Klosterdatenbank-to-FactGrid-Workflow, which is to create Items for religious communities.

As the name suggests, the "Monastery" is the central unit of the monastery database. In the table `gs_monastery`, each row represents a religious community. This is also reflected in the query options of the web interface of the monastery database. Applying the various filter functions always results in a list of religious communities. In the detail view, all relevant information from linked tables is then displayed. Here, the religious community is always at the center. All further information is displayed in connection with the religious community. By integrating with FactGrid, the query options are expanded. For example, it is now possible to query only building complexes. At the same time, the structure of the data model must be taken into account when querying information, such as the geographical location of a religious community at a specific point in time.

In order to import the religious communities to FactGrid, the following workflow will create labels based on the monastery name and it's translation from Notebook 1a. Other than the monastery locations/building complexes, the religious communities are connected to a series of external identifiers, which will party be transferred to FactGrid.

## Preparation
The notebook requires the following libraries to run. If an error occurs, make sure the libraries are installed on your system.

In [11]:
import pandas as pd
import numpy as np

First, the export files are loaded into [Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataframes are stored in a dictionary with the keys as the filenames, for easier access.

In [12]:
# Load Access exports
from helper_functions import load_files_from_folder, query_factgrid

export_files = load_files_from_folder("data/exports_monasteryDB", "xlsx")

# Create dataframes for each table
dataframes = {key: pd.read_excel(value) for key, value in export_files.items()}

# Add dataframe for monasteries in factGrid (stored in a different directory)
dataframes["building_complexes_in_factgrid"] = query_factgrid("building_complexes")
dataframes["monasteries_in_factgrid"] = query_factgrid("monasteries")
# Add translation data
dataframes["translated"] = pd.read_csv("data/translation/translated.csv")

The next cell prepares the datasets for the workflow. First, the table `gs_monastery` is filtered for those religious communities that have the status "Online". This means, that the datasets are considered finished and are no longer being actively worked on. Afterwards, only the column `gsn_id` and `monastery_name` are selected. Finally, the column `gsn_id` is filtered against the list of monasteries with monastery database identifiers in FactGrid (`factgrid_data/monasteries_in_factgrid.xlsx`) to make sure that no duplicates are produced.

In [13]:
# Filter for monasteries online
monasteries_online = dataframes["gs_monastery"][dataframes["gs_monastery"]["status"] == "Online"]
# Drop irrelevant columns
prepared_df = monasteries_online[["id_gsn", "monastery_name", "patrocinium"]]
print(f'{len(prepared_df[prepared_df["id_gsn"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"].astype(int))])} monasteries already exist in FactGrid and are filtered out')
prepared_df = prepared_df[~prepared_df["id_gsn"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"].astype(int))]
prepared_df

39 monasteries already exist in FactGrid and are filtered out


,id_gsn,monastery_name,patrocinium
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN
1,4647,Kapuzinerkloster Ettlingen,NaN
2,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina
3,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus
5,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt
...,...,...,...
494,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae
495,10407,"Benediktinerkloster Genolier, Schweiz",Maria
496,60132,Franziskanerterziaren Herrieden,NaN
498,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes


## Labels

The label in FactGrid will be the `monastery_name`.

In [14]:
prepared_df["Lde"] = prepared_df["monastery_name"]
prepared_df

,id_gsn,monastery_name,patrocinium,Lde
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"Augustinerchorherrenstift, später Domstift Wie..."
1,4647,Kapuzinerkloster Ettlingen,NaN,Kapuzinerkloster Ettlingen
2,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"Franziskanertertiarinnenkloster St. Catharina,..."
3,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,Kapuzinerkloster Gößweinstein (später Franzisk...
5,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"Franziskanertertiarinnenkloster Roermond, Nied..."
...,...,...,...,...
494,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,Kapuzinerkloster Freiburg
495,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"Benediktinerkloster Genolier, Schweiz"
496,60132,Franziskanerterziaren Herrieden,NaN,Franziskanerterziaren Herrieden
498,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied..."


In [15]:
gs_alt_label_monastery = dataframes["gs_alt_label_monastery"]
gs_alt_label_monastery = gs_alt_label_monastery[gs_alt_label_monastery["gsn_id"].isin(prepared_df["id_gsn"])]
for index, row in gs_alt_label_monastery.iterrows():
    if f'L{row["lang"]}'not in prepared_df.columns.tolist() or pd.isna(prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], f'L{row["lang"]}'].values[0]):
        prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], f'L{row["lang"]}'] = f'\"{row["alt_label_monastery"]}\"' if not pd.isna(row["alt_label_monastery"]) else np.nan
    else:
        prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], f'A{row["lang"]}'] = f'\"{row["alt_label_monastery"]}\"' if not pd.isna(row["alt_label_monastery"]) else np.nan
prepared_df

,id_gsn,monastery_name,patrocinium,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,Lfr,Let,Lit,Lcs,Afr
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"Augustinerchorherrenstift, später Domstift Wie...","""eccl. Nove civit.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4647,Kapuzinerkloster Ettlingen,NaN,Kapuzinerkloster Ettlingen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"Franziskanertertiarinnenkloster St. Catharina,...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,Kapuzinerkloster Gößweinstein (später Franzisk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"Franziskanertertiarinnenkloster Roermond, Nied...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,Kapuzinerkloster Freiburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"Benediktinerkloster Genolier, Schweiz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,60132,Franziskanerterziaren Herrieden,NaN,Franziskanerterziaren Herrieden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


As the religious communities should also have an english label, the monastery name is automatically translated. The file `translated.csv` that has been created by notebook 1a - Translation can be reused for this. This means, that if you already ran Notebooks 1 - Building Complexes and 1a - translation completely, you don't have to do anything exept running this notebook to create the monasteries. 

In [16]:
prepared_df = pd.merge(prepared_df, dataframes["translated"], how="left", left_on="Lde", right_on="monastery_Lde").drop_duplicates()
if not "Len" in prepared_df.columns:
    prepared_df["Len"] = np.nan
prepared_df["Len"].fillna(prepared_df["monastery_Len"], inplace=True)
prepared_df["Lde"] = prepared_df["Lde"].apply(lambda x: f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df["Len"] = prepared_df["Len"].str.strip("\"").apply(lambda x: f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df.drop(columns=[f"{x}_{y}" for x in ["monastery", "building"] for y in ["Lde", "Dde", "Len", "Den"]], inplace=True)
prepared_df.drop_duplicates(subset="id_gsn", inplace=True)
prepared_df

C:\Users\danielzik1\AppData\Local\Temp\ipykernel_25200\289695188.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prepared_df["Len"].fillna(prepared_df["monastery_Len"], inplace=True)


,id_gsn,monastery_name,patrocinium,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,Lfr,Let,Lit,Lcs,Afr
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"""Augustinerchorherrenstift, später Domstift Wi...","""eccl. Nove civit.""",NaN,"""Canons Regular of St Augustine, later Cathedr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4647,Kapuzinerkloster Ettlingen,NaN,"""Kapuzinerkloster Ettlingen""",NaN,NaN,"""Capuchin friary Ettlingen""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"""Franziskanertertiarinnenkloster St. Catharina...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,"""Kapuzinerkloster Gößweinstein (später Franzis...",NaN,NaN,"""Capuchin friary Gößweinstein (later Francisca...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"""Franziskanertertiarinnenkloster Roermond, Nie...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,"""Kapuzinerkloster Freiburg""",NaN,NaN,"""Capuchin friary Freiburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"""Benediktinerkloster Genolier, Schweiz""",NaN,NaN,"""Benedictine monastery Genolier, Switzerland""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,60132,Franziskanerterziaren Herrieden,NaN,"""Franziskanerterziaren Herrieden""",NaN,NaN,"""Franciscans Herrieden""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"""Kollegiatstift Sint-Janshuis, Amersfoort, Nie...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Patrocinium
As of now, the monastery database only contains non-standardized data about the dedication of the monasteries. It can be found in the table `gs_monastery`, field `patrocinium`. The column contains simple string data, expressing one or more patrons of the monastery. For example, the Franciscan Monastery of Braunschweig (GSN [60](https://klosterdatenbank.adw-goe.de/gsn/60)) is dedicated to "Maria, Franziskus, Benward". As sorting and identifying the patrocinia would be a research project in itself, we decided to just add the simple string expressions. Even this reduced information can be helpful to identify monasteries in historical sources, as they are often times referenced by their patrocinium. In FactGrid, the Property [P662](https://database.factgrid.de/wiki/Property:P662) was introduced as a string equivalent to [P391](https://database.factgrid.de/wiki/Property:P391) to enable the string representation of patrociniums in FactGrid.

In [17]:
prepared_df["P662"] = prepared_df["patrocinium"]
prepared_df["P662"].replace("unbekannt", np.nan, inplace=True)
prepared_df["P662"] = prepared_df["P662"].apply(lambda x: f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df["S471"] = prepared_df["id_gsn"].apply(lambda x: f'\"{x}\"')
prepared_df

C:\Users\danielzik1\AppData\Local\Temp\ipykernel_25200\1737197524.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prepared_df["P662"].replace("unbekannt", np.nan, inplace=True)


,id_gsn,monastery_name,patrocinium,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,Lfr,Let,Lit,Lcs,Afr,P662,S471
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"""Augustinerchorherrenstift, später Domstift Wi...","""eccl. Nove civit.""",NaN,"""Canons Regular of St Augustine, later Cathedr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""4080"""
3,4647,Kapuzinerkloster Ettlingen,NaN,"""Kapuzinerkloster Ettlingen""",NaN,NaN,"""Capuchin friary Ettlingen""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""4647"""
4,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"""Franziskanertertiarinnenkloster St. Catharina...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Catharina""","""8217"""
5,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,"""Kapuzinerkloster Gößweinstein (später Franzis...",NaN,NaN,"""Capuchin friary Gößweinstein (later Francisca...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Franziskus""","""60342"""
6,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"""Franziskanertertiarinnenkloster Roermond, Nie...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""8534"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,"""Kapuzinerkloster Freiburg""",NaN,NaN,"""Capuchin friary Freiburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Unbefleckte Empfängnis Mariae""","""20642"""
506,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"""Benediktinerkloster Genolier, Schweiz""",NaN,NaN,"""Benedictine monastery Genolier, Switzerland""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Maria""","""10407"""
507,60132,Franziskanerterziaren Herrieden,NaN,"""Franziskanerterziaren Herrieden""",NaN,NaN,"""Franciscans Herrieden""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""60132"""
508,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"""Kollegiatstift Sint-Janshuis, Amersfoort, Nie...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Johannes""","""8016"""


## Link to Germania Sacra and monastery database

Every religious community is linked to the corresponding ID from the monastery database using the existing Property [P471](https://database.factgrid.de/wiki/Property:P471). Also, to state which research project contributed to the dataset, the project-item "Germania Sacra in FactGrid" ([Q153178](https://database.factgrid.de/wiki/Item:Q153178)) is linked using property [P131](https://database.factgrid.de/wiki/Property:P131).

In [18]:
prepared_df["P471"] = prepared_df["id_gsn"].apply(lambda x: f'\"{x}\"')
prepared_df["P131"] = "Q153178"
prepared_df

,id_gsn,monastery_name,patrocinium,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,Lfr,Let,Lit,Lcs,Afr,P662,S471,P471,P131
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"""Augustinerchorherrenstift, später Domstift Wi...","""eccl. Nove civit.""",NaN,"""Canons Regular of St Augustine, later Cathedr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""4080""","""4080""",Q153178
3,4647,Kapuzinerkloster Ettlingen,NaN,"""Kapuzinerkloster Ettlingen""",NaN,NaN,"""Capuchin friary Ettlingen""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""4647""","""4647""",Q153178
4,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"""Franziskanertertiarinnenkloster St. Catharina...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Catharina""","""8217""","""8217""",Q153178
5,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,"""Kapuzinerkloster Gößweinstein (später Franzis...",NaN,NaN,"""Capuchin friary Gößweinstein (later Francisca...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Franziskus""","""60342""","""60342""",Q153178
6,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"""Franziskanertertiarinnenkloster Roermond, Nie...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""8534""","""8534""",Q153178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,"""Kapuzinerkloster Freiburg""",NaN,NaN,"""Capuchin friary Freiburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Unbefleckte Empfängnis Mariae""","""20642""","""20642""",Q153178
506,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"""Benediktinerkloster Genolier, Schweiz""",NaN,NaN,"""Benedictine monastery Genolier, Switzerland""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Maria""","""10407""","""10407""",Q153178
507,60132,Franziskanerterziaren Herrieden,NaN,"""Franziskanerterziaren Herrieden""",NaN,NaN,"""Franciscans Herrieden""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""60132""","""60132""",Q153178
508,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"""Kollegiatstift Sint-Janshuis, Amersfoort, Nie...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Johannes""","""8016""","""8016""",Q153178


## External Identifiers

Other than for the building complexes, the monastery database provides a range of external URLs for each religious community to connect to existing databases and online ressources, such as wikipedia or wikidata. Some of these identifiers should also be included in FactGrid. The table `gs_external_urls_monastery` contains a mapping between religious communities, types of external identifiers and the specific identifiers that can be used to find the ressource in the corresponfing system. The table `gs_external_url_type_with_factgrid` contains information on how to resolve the identifiers using base URLs and also contains a mapping between URL types and existing FactGrid Identifiers. There are three different cases to consider: A URL type can be linked in FactGrid using a property that was introduced by the community such as the GND-ID ([P76](https://database.factgrid.de/wiki/Property:P76)). Links to other communities within the Wiki-Infrastructure are handled using Sitelinks. For Quickstatements they would be referenced by the letter "S" followed by the short name for the wiki-project. For example, to state the Q-Number for an item in Wikidata, the command would be `Swikidatawiki`, with the `S` standing for "sitelink" and `wikidatawiki` the short name for Wikidata. Third, there are also cases in which an Identifier has no correspondant Property in FactGrid. In these cases, the information is omitted. However, it can always be referenced using the monastery database's original interface which is linked in the references of each statement and via the corresponding identifier-property in FactGrid. 

The following cell processes the information on external URLs and adds them to the table. Please not that in order for the Sitelinks to be imported correctly, the final CSV table has to be processed with the function `df_to_qs_v1` from `helper_functions.py` as done in the cells below.

In [19]:
gs_external_url_type_with_factgrid = dataframes["gs_external_url_type_with_factgrid"].dropna(subset="factgrid_property")
url_factgrid = pd.merge(dataframes["gs_external_urls_monastery"], gs_external_url_type_with_factgrid, how="left", left_on="url_type_id", right_on="id_url_type")[["url_name_formatter","gsn_id", "url_value", "factgrid_property"]].dropna(subset="factgrid_property")
for index, row in url_factgrid.iterrows():
    if row["gsn_id"] in prepared_df["id_gsn"].values and not "Wikipedia-Artikel zum Baudenkmal" in row["url_name_formatter"]:
        prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], row["factgrid_property"]] = f'\"{row["url_value"]}\"'
prepared_df

,id_gsn,monastery_name,patrocinium,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,...,P76,Swikidatawiki,Sdewiki,P378,Snlwiki,Sfrwiki,Setwiki,Scswiki,Splwiki,Senwiki
0,4080,"Augustinerchorherrenstift, später Domstift Wie...",NaN,"""Augustinerchorherrenstift, später Domstift Wi...","""eccl. Nove civit.""",NaN,"""Canons Regular of St Augustine, later Cathedr...",NaN,NaN,NaN,...,"""1052446426""","""Q111439337""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4647,Kapuzinerkloster Ettlingen,NaN,"""Kapuzinerkloster Ettlingen""",NaN,NaN,"""Capuchin friary Ettlingen""",NaN,NaN,NaN,...,NaN,"""Q107762079""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8217,"Franziskanertertiarinnenkloster St. Catharina,...",Catharina,"""Franziskanertertiarinnenkloster St. Catharina...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,...,NaN,"""Q107761488""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,60342,Kapuzinerkloster Gößweinstein (später Franzisk...,Franziskus,"""Kapuzinerkloster Gößweinstein (später Franzis...",NaN,NaN,"""Capuchin friary Gößweinstein (later Francisca...",NaN,NaN,NaN,...,"""16334155-2""","""Q1775294""","""Kloster_Gößweinstein""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8534,"Franziskanertertiarinnenkloster Roermond, Nied...",unbekannt,"""Franziskanertertiarinnenkloster Roermond, Nie...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,20642,Kapuzinerkloster Freiburg,Unbefleckte Empfängnis Mariae,"""Kapuzinerkloster Freiburg""",NaN,NaN,"""Capuchin friary Freiburg""",NaN,NaN,NaN,...,"""102755847X""","""Q28977325""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,10407,"Benediktinerkloster Genolier, Schweiz",Maria,"""Benediktinerkloster Genolier, Schweiz""",NaN,NaN,"""Benedictine monastery Genolier, Switzerland""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,60132,Franziskanerterziaren Herrieden,NaN,"""Franziskanerterziaren Herrieden""",NaN,NaN,"""Franciscans Herrieden""",NaN,NaN,NaN,...,NaN,"""Q1775337""","""Kloster_Herrieden""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,8016,"Kollegiatstift Sint-Janshuis, Amersfoort, Nied...",Johannes,"""Kollegiatstift Sint-Janshuis, Amersfoort, Nie...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,...,NaN,"""Q107757457""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Finalizing
Before linking the monasteries to their orders and building complexes, they need to have a Q-number. Load the table that is created below into Quickstatements and save the created monasteries as specified.



In [20]:
from helper_functions import df_to_qs_v1
prepared_df["id_gsn"].to_csv("data/intermediate_results/new_monasteries_ids.csv")

monastery_upload = prepared_df.drop(columns=["id_gsn", "monastery_name", "patrocinium"])
monastery_upload.insert(0, "qid", np.nan)
monastery_upload.to_excel("data/results/monasteries/import_monasteries.xlsx")
monastery_upload.to_csv("data/results/monasteries/import_monasteries.csv")
with open("data/results/monasteries/import_monasteries.tsv", "w") as file:
    file.write(df_to_qs_v1(monastery_upload))
monastery_upload

,qid,Lde,Lla,Ade,Len,Lnl,Lpl,Anl,Lfr,Let,...,P76,Swikidatawiki,Sdewiki,P378,Snlwiki,Sfrwiki,Setwiki,Scswiki,Splwiki,Senwiki
0,NaN,"""Augustinerchorherrenstift, später Domstift Wi...","""eccl. Nove civit.""",NaN,"""Canons Regular of St Augustine, later Cathedr...",NaN,NaN,NaN,NaN,NaN,...,"""1052446426""","""Q111439337""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"""Kapuzinerkloster Ettlingen""",NaN,NaN,"""Capuchin friary Ettlingen""",NaN,NaN,NaN,NaN,NaN,...,NaN,"""Q107762079""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"""Franziskanertertiarinnenkloster St. Catharina...",NaN,NaN,"""Tertiaries St. Catherine, Haarlem""","""Tertiarissen St. Catharina, Haarlem""",NaN,NaN,NaN,NaN,...,NaN,"""Q107761488""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,"""Kapuzinerkloster Gößweinstein (später Franzis...",NaN,NaN,"""Capuchin friary Gößweinstein (later Francisca...",NaN,NaN,NaN,NaN,NaN,...,"""16334155-2""","""Q1775294""","""Kloster_Gößweinstein""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,"""Franziskanertertiarinnenkloster Roermond, Nie...",NaN,NaN,"""Tertiaries Roermond""","""Tertiarissen Godsweerd, Roermond""",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NaN,"""Kapuzinerkloster Freiburg""",NaN,NaN,"""Capuchin friary Freiburg""",NaN,NaN,NaN,NaN,NaN,...,"""102755847X""","""Q28977325""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,NaN,"""Benediktinerkloster Genolier, Schweiz""",NaN,NaN,"""Benedictine monastery Genolier, Switzerland""",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,NaN,"""Franziskanerterziaren Herrieden""",NaN,NaN,"""Franciscans Herrieden""",NaN,NaN,NaN,NaN,NaN,...,NaN,"""Q1775337""","""Kloster_Herrieden""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,NaN,"""Kollegiatstift Sint-Janshuis, Amersfoort, Nie...",NaN,NaN,"""Canons Amersfoort""","""Kanunniken Sint-Janshuis, Amersfoort""",NaN,NaN,NaN,NaN,...,NaN,"""Q107757457""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
